In [28]:
import pandas as pd
import numpy as np

In [29]:
df = pd.read_hdf('data/processed/ach_at_full.h5')

In [21]:
neighbours = [1, 2, 3]
subset = ['t', 'window_id', 4] + neighbours
df2 = df[subset]
df2.head()

,t,window_id,4,1,2,3
0,1.0,0,-124.371209,-126.704873,-129.462254,-121.519825
1,2.0,0,-124.485424,-126.796326,-129.559494,-121.627592
2,3.0,0,-124.598914,-126.887027,-129.655970,-121.734591
3,4.0,0,-124.711680,-126.976977,-129.751683,-121.840824
4,5.0,0,-124.823721,-127.066176,-129.846632,-121.946291


In [23]:
df_neighbours = df[neighbours]
df_neighbours.head()

,1,2,3
0,-126.704873,-129.462254,-121.519825
1,-126.796326,-129.559494,-121.627592
2,-126.887027,-129.655970,-121.734591
3,-126.976977,-129.751683,-121.840824
4,-127.066176,-129.846632,-121.946291


In [11]:
df_diffs = (df[neighbours]
.add_suffix('_diff'))

In [24]:
df_neighbours.sub(df[4], axis=0)

,1,2,3
0,-2.333664,-5.091045,2.851384
1,-2.310902,-5.074070,2.857832
2,-2.288113,-5.057056,2.864323
3,-2.265297,-5.040003,2.870856
4,-2.242455,-5.022911,2.877430
...,...,...,...
179995,5.770409,4.123739,3.186573
179996,5.771933,4.126524,3.201537
179997,5.773412,4.129286,3.216494
179998,5.774848,4.132025,3.231442


In [30]:
col = 4
df_diffs = (df[neighbours]
           .sub(df[col], axis=0)
           .add_suffix('_'+str(col)+'_diff'))
df2 = pd.concat([df[['t','window_id']], df_diffs], axis=1)

In [37]:
print(df2.head())
df[0]

     t  window_id  1_4_diff  2_4_diff  3_4_diff
0  1.0          0 -2.333664 -5.091045  2.851384
1  2.0          0 -2.310902 -5.074070  2.857832
2  3.0          0 -2.288113 -5.057056  2.864323
3  4.0          0 -2.265297 -5.040003  2.870856
4  5.0          0 -2.242455 -5.022911  2.877430


0        -135.904409
1        -135.970862
2        -136.036568
3        -136.101527
4        -136.165739
             ...    
179995     -3.811497
179996     -3.735131
179997     -3.658896
179998     -3.582794
179999     -3.506826
Name: 0, Length: 2700000, dtype: float64